#### Grupo 2: Ziteng Huang, Lubin Ye, Jiahui You

#### Breve memoria
En nuestro algoritmo genético, para la generación de una nueva generación empleamos selección por torneo, luego cruce y mutación. Utilizamos una población inicial de 100 cromosomas (de tamaño 900) y generamos así 50 generaciones. El mejor resultado obtenido con esta configuración y parámetros es de 419084033 de oro en 5 segundos.

In [9]:
import random

class ProblemaGenetico(object):
        def __init__(self, genes, fun_dec, fun_muta, fun_cruza, fun_fitness, longitud_individuos):
            self.genes = genes
            self.fun_dec = fun_dec
            self.fun_cruza = fun_cruza
            self.fun_muta = fun_muta
            self.fun_fitness = fun_fitness
            self.longitud_individuos = longitud_individuos
            """Constructor de la clase"""
                
        def decodifica(self, genotipo):
            """Devuelve el fenotipo a partir del genotipo"""
            fenotipo = self.fun_dec(genotipo)
            return fenotipo
        def muta(self, cromosoma,prob):
            """Devuelve el cromosoma mutado"""   
            mutante = self.fun_muta(cromosoma,prob)
            return mutante
        def cruza(self, cromosoma1, cromosoma2):         
            """Devuelve el cruce de un par de cromosomas"""
            cruce = self.fun_cruza(cromosoma1,cromosoma2)
            return cruce 
        def fitness(self, cromosoma):    
            """Función de valoración"""
            valoracion = self.fun_fitness(cromosoma)
            return valoracion
        
operation = {
     0 : {
        'cost' : 0,
        'oropsec' : 0
     },
     1 : {
        'cost' : 15,
        'oropsec' : 1
     },
     2 : {
        'cost' : 100,
        'oropsec' : 5
     },
     3 : {
        'cost' : 300,
        'oropsec' : 20
     },
     4 : {
        'cost' : 2000,
        'oropsec' : 100
     },
     5 : {
        'cost' : 15000,
        'oropsec' : 700
     },
     6 : {
        'cost' : 250000,
        'oropsec' : 10000
     }
}

In [10]:
def fun_dec(cromosoma):
    oro = 0
    oropsec = 1

    for i in range(len(cromosoma)):
        oro += oropsec
        if oro >= operation[cromosoma[i]]["cost"]:
            oro -= operation[cromosoma[i]]["cost"]
            oropsec += operation[cromosoma[i]]["oropsec"]
        else:
            cromosoma[i] = 0
    return oro, cromosoma

def fun_mutar(cromosoma,prob):
    """Elige un elemento al azar del cromosoma y lo modifica con una probabilidad igual a prob"""
    l = len(cromosoma)
    p = random.randint(0,l-1)
    if prob > random.uniform(0,1):
        cromosoma[p] = random.randint(0,6)
    return cromosoma

def fun_cruzar(cromosoma1, cromosoma2):
    """Cruza los cromosomas por la mitad (podemos cambiar la función de cruce eligiendo otro punto de cruce al azar)"""
    l1 = len(cromosoma1)
    l2 = len(cromosoma2)
    cruce1 = cromosoma1[0:l1//2]+cromosoma2[l1//2:l2]
    cruce2 = cromosoma2[0:l2//2]+cromosoma1[l2//2:l1]
    return [cruce1,cruce2]

def fun_fitness(cromosoma):
    oro = 0
    oropsec = 1

    for i in range(len(cromosoma)):
        oro += oropsec
        if oro >= operation[cromosoma[i]]["cost"]:
            oro -= operation[cromosoma[i]]["cost"]
            oropsec += operation[cromosoma[i]]["oropsec"]
        else:
            cromosoma[i] = 0

    return oro

In [11]:
def poblacion_inicial(problema_genetico, size):
    l=[] # población inicial
    for i in range(size): # añadimos a la población size individuos
        x=[0,0,0,0,0,0,0,0,0,0,0,0,0,0] # 14 primeros genes a 0
        for j in range(problema_genetico.longitud_individuos - 14): # los individuos se generan eligiendo sus genes
                                                               # de manera eleatoria de entre los genes posibles
            x.append(random.choice(problema_genetico.genes))
        l.append(x) 
    return l

def cruza_padres(problema_genetico,padres):
    l = []
    for i in range(len(padres)//2):# asumimos que la población de la que partimos tiene tamaño par
        desc = problema_genetico.fun_cruza(padres[2*i],padres[2*i+1]) # El cruce se realiza con la función de cruce  
                                                                     # proporcionada por el propio problema genético
        l.append(desc[0]) # La población resultante se obtiene de cruzar el padre[0] con padre[1], padre[2] con padre[3]...
        l.append(desc[1]) # y añadir cada par de descendientes a la nueva población
    return l

def muta_individuos(problema_genetico, poblacion, prob):
    # problema_genetico.muta(x,prob) para todos los individuos de la poblacion.
    l = []
    for i in poblacion:
        l.append(problema_genetico.muta(i,prob))
    return l

def seleccion_por_torneo(problema_genetico, poblacion, n, k, opt):
    """Selección por torneo de n individuos de una población. Siendo k el nº de participantes
        y opt la función max o min."""
    seleccionados = []
    for i in range(n):
        participantes = random.sample(poblacion,k) #obtener numero k de individuos de la poblacion sin repeticion
        seleccionado = opt(participantes, key=problema_genetico.fitness)
        seleccionados.append(seleccionado)
        # poblacion.remove(seleccionado)
    return seleccionados  

def nueva_generacion(problema_genetico, k,opt, poblacion, n_padres, n_directos, prob_mutar):
    padres2 = seleccion_por_torneo(problema_genetico, poblacion, n_directos, k,opt) 
    padres1 = seleccion_por_torneo(problema_genetico, poblacion, n_padres , k, opt)
    cruces =  cruza_padres(problema_genetico,padres1)
    generacion = padres2+cruces
    resultado_mutaciones = muta_individuos(problema_genetico, generacion, prob_mutar)
    return resultado_mutaciones

def algoritmo_genetico(problema_genetico,k,opt,ngen,size,prop_cruces,prob_mutar):
    poblacion= poblacion_inicial(problema_genetico,size)
    n_padres=round(size*prop_cruces)
    n_padres= int (n_padres if n_padres%2==0 else n_padres-1)
    n_directos = size-n_padres
    for _ in range(ngen):
        poblacion= nueva_generacion(problema_genetico,k,opt,poblacion,n_padres, n_directos,prob_mutar)

    mejor_cr= opt(poblacion, key=problema_genetico.fitness)
    mejor_valor, mejor_cr=problema_genetico.decodifica(mejor_cr)
    return (mejor_valor,mejor_cr) 

In [14]:
genes = [0,1,2,3,4,5,6]
tiempo = 900
num_gen = 50
poblacion_ini = 100

idle=ProblemaGenetico(genes, fun_dec, fun_mutar, fun_cruzar, fun_fitness, tiempo)

salida=algoritmo_genetico(idle,4,max,num_gen,poblacion_ini,0.7,0.1) # variante que devuelve salida 

print(f"Oro Maximo = {salida[0]}\nSolucion Obtenida:\n{salida[1]}")

Oro Maximo = 419084033
Solucion Obtenida:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 1, 2, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 3, 0, 1, 0, 0, 0, 1, 3, 1, 0, 1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 2, 0, 3, 3, 2, 0, 0, 4, 0, 0, 0, 2, 0, 0, 3, 0, 2, 2, 0, 4, 3, 3, 0, 0, 4, 1, 0, 2, 4, 1, 1, 0, 4, 0, 1, 0, 3, 3, 2, 0, 0, 1, 3, 1, 0, 1, 0, 0, 0, 2, 2, 1, 5, 2, 2, 0, 2, 4, 1, 1, 2, 0, 0, 1, 3, 0, 5, 4, 4, 1, 3, 2, 0, 1, 5, 4, 2, 4, 3, 0, 4, 0, 2, 5, 5, 1, 1, 5, 3, 0, 3, 3, 3, 1, 0, 3, 2, 0, 0, 1, 0, 1, 3, 5, 0, 4, 5, 0, 4, 5, 0, 2, 5, 3, 3, 2, 5, 3, 2, 4, 1, 2, 0, 4, 5, 4, 4, 5, 0, 0, 2, 1, 0, 5, 6, 5, 4, 5, 5, 0, 0, 0, 3, 0, 2, 1, 3, 4, 3, 3, 3, 3, 1, 6, 0, 0, 0, 4, 6, 2, 5, 0, 0, 2, 1, 5, 3, 2, 2, 4, 4, 6, 0, 5, 3, 5, 0, 4, 6, 0, 3, 5, 2, 5, 